In [2]:
import numpy as np
import h5py
from scipy.sparse import linalg as lg
from utils import ed
from utils import tfim
from utils import misc
from utils import calc

data_dir = "/home/stavros/DATA"

Create `.h5` by calculating quantities (observables & overlaps) on simulated and exact wavefunctions

In [2]:
n_sites = 6
h_init = 1.0
h_ev = 0.5

T_list = [20, 25, 30, 35, 40, 45, 50]
t_final = 1.0

ham = tfim.tfim_hamiltonian(n_sites, h=h_ev)
pauli = misc.Pauli()

get_norm = lambda state: (np.abs(state)**2).sum(axis=1)
get_clock_energy = lambda state, clock: (state.ravel().conj().dot(clock.toarray().dot(state.ravel())) 
                                         / (np.abs(state)**2).sum())
get_sigma_x = lambda state: calc.ev_local(state, pauli.X)


overlaps = {"composite": [], "avg": [], "time": []}
metrics = {"clock/norm": [], "clock/sigma_x": [], "clock/clock_energy": [],
           "exact/norm": [], "exact/sigma_x": [], "exact/clock_energy": []}
for time_steps in T_list:
    t = np.linspace(0.0, t_final, time_steps + 1)
    dt = t[1] - t[0]
        
    # Calculate exact evolution
    exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)
        
    # Load Clock evolution
    clock = ed.construct_sparse_clock(ham, dt, time_steps)
    clock_state = np.load("{}/ClockV3/final_dense/sweep_fullwv_N{}M{}.npy".format(data_dir, n_sites, time_steps))
    print("T = {}".format(time_steps))
    
    # Calculate overlaps
    overlaps["time"].append(calc.time_overlap(exact_state, clock_state))
    overlaps["avg"].append(overlaps["time"][-1].mean())
    overlaps["composite"].append(calc.overlap(exact_state, clock_state))
    
    # Calculate metrics
    for key, state in zip(["exact", "clock"], [exact_state, clock_state]):
        metrics["{}/norm".format(key)].append(get_norm(state))
        metrics["{}/clock_energy".format(key)].append(get_clock_energy(state, clock))
        metrics["{}/sigma_x".format(key)].append(get_sigma_x(state))
        
        
# Save so that you don't have to run the above all the time.
import h5py
file = h5py.File("sweep_fullwv_tf1_N{}.h5".format(n_sites), "w")
file["T_list"] = T_list

file["overlaps/composite"] = overlaps["composite"]
file["overlaps/avg"] = overlaps["avg"]
file["metrics/clock/clock_energy"] = metrics["clock/clock_energy"]
file["metrics/exact/clock_energy"] = metrics["exact/clock_energy"]

for i in range(len(T_list)):
    file["overlaps/time/{}".format(i)] = overlaps["time"][i]
    file["metrics/clock/norm/{}".format(i)] = metrics["clock/norm"][i]
    file["metrics/exact/norm/{}".format(i)] = metrics["exact/norm"][i]
    file["metrics/clock/sigma_x/{}".format(i)] = metrics["clock/sigma_x"][i]
    file["metrics/exact/sigma_x/{}".format(i)] = metrics["exact/sigma_x"][i]

file.close()

T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50


Combines many `.h5`s to a big `.h5`

In [ ]:
new_file = h5py.File("master_tf1_N6.h5", "w")

file = h5py.File("{}/ClockV3/observables/fullwv_tf1_N6.h5".format(data_dir))
new_file["T_list"] = file["T_list"][()]
new_file["fullwv/overlaps/composite"] = file["overlaps/composite"][()]
new_file["fullwv/overlaps/avg"] = file["overlaps/avg"][()]
new_file["fullwv/clock_energy"] = file["metrics/clock/clock_energy"][()]
new_file["exact/clock_energy"] = file["metrics/exact/clock_energy"][()]

for i, time_steps in enumerate(file["T_list"][()]):
    t = np.linspace(0.0, t_final, time_steps + 1)
    exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)
    new_file["exact/state/{}".format(i)] = exact_state
    new_file["fullwv/state/{}".format(i)] = np.load(
        "{}/ClockV2/final_dense/allstates_fullwv_N6M{}.npy".format(data_dir, time_steps))

    new_file["fullwv/overlaps/time/{}".format(i)] = file["overlaps/time/{}".format(i)][()]
    
    new_file["exact/norm/{}".format(i)] = file["metrics/exact/norm/{}".format(i)][()]
    new_file["fullwv/norm/{}".format(i)] = file["metrics/clock/norm/{}".format(i)][()]
    new_file["exact/sigma_x/{}".format(i)] = file["metrics/exact/sigma_x/{}".format(i)][()]
    new_file["fullwv/sigma_x/{}".format(i)] = file["metrics/clock/sigma_x/{}".format(i)][()]
file.close()


for D in [2, 3, 4, 5, 6]:
    file = h5py.File("{}/ClockV3/observables/mpsD{}_tf1_N6.h5".format(data_dir, D))
    tvmc_file = h5py.File("{}/ClockV3/observables/tvmc_mpsD{}_tf1_N6.h5".format(data_dir, D))
        
    new_file["mpsD{}/overlaps/composite".format(D)] = file["overlaps/composite"][()]
    new_file["mpsD{}/overlaps/avg".format(D)] = file["overlaps/avg"][()]
    new_file["mpsD{}/clock_energy".format(D)] = file["metrics/clock/clock_energy"][()]
    
    new_file["tvmc_mpsD{}/overlaps/composite".format(D)] = tvmc_file["overlaps/composite"][()]
    new_file["tvmc_mpsD{}/overlaps/avg".format(D)] = tvmc_file["overlaps/avg"][()]
    new_file["tvmc_mpsD{}/clock_energy".format(D)] = tvmc_file["metrics/clock/clock_energy"][()]
    
    for i, time_steps in enumerate(file["T_list"][()]):
        new_file["mpsD{}/state/{}".format(D, i)] = np.load(
            "{}/ClockV2/final_dense/allstates_mpsD{}_N6M{}.npy".format(data_dir, D, time_steps))
        new_file["tvmc_mpsD{}/state/{}".format(D, i)] = np.load(
            "{}/ClockV2/final_dense/tvmc/tvmc_allstates_stepmpsD{}_N6M{}.npy".format(data_dir, D, time_steps))

        new_file["mpsD{}/overlaps/time/{}".format(D, i)] = file["overlaps/time/{}".format(i)][()]
        new_file["mpsD{}/norm/{}".format(D, i)] = file["metrics/clock/norm/{}".format(i)][()]
        new_file["mpsD{}/sigma_x/{}".format(D, i)] = file["metrics/clock/sigma_x/{}".format(i)][()]
        
        new_file["tvmc_mpsD{}/overlaps/time/{}".format(D, i)] = tvmc_file["overlaps/time/{}".format(i)][()]
        new_file["tvmc_mpsD{}/norm/{}".format(D, i)] = tvmc_file["metrics/clock/norm/{}".format(i)][()]
        new_file["tvmc_mpsD{}/sigma_x/{}".format(D, i)] = tvmc_file["metrics/clock/sigma_x/{}".format(i)][()]
    
    file.close()
    tvmc_file.close()
    
new_file.close()

In [5]:
file.close()

In [9]:
n_sites = 6
h_init = 1.0
h_ev = 0.5

T_list = [20, 25, 30, 35, 40, 45, 50]
t_final = 1.0

ham = tfim.tfim_hamiltonian(n_sites, h=h_ev)
pauli = misc.Pauli()

get_norm = lambda state: (np.abs(state)**2).sum(axis=1)
get_clock_energy = lambda state, clock: (state.ravel().conj().dot(clock.toarray().dot(state.ravel())) 
                                         / (np.abs(state)**2).sum())
get_sigma_x = lambda state: calc.ev_local(state, pauli.X)

file = h5py.File("{}/ClockV3/everything_tf1N{}.h5".format(data_dir, n_sites), "a")
overlaps = {"avg": [], "composite": []}
clock_energy = []
for i, time_steps in enumerate(T_list):
    t = np.linspace(0.0, t_final, time_steps + 1)
    dt = t[1] - t[0]
    # Calculate exact evolution
    exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)
        
    clock = ed.construct_sparse_clock(ham, dt, time_steps)
    clock_state = np.load("{}/ClockV3/final_dense/sweep_fullwv_N{}M{}.npy".format(data_dir, n_sites, time_steps))
    time_overlap = calc.time_overlap(exact_state, clock_state)
    overlaps["avg"].append(time_overlap.mean())
    overlaps["composite"].append(calc.overlap(exact_state, clock_state))
    clock_energy.append(get_clock_energy(clock_state, clock))
    
    file["sweep_fullwv/overlaps/time/{}".format(i)] = time_overlap
    file["sweep_fullwv/state/{}".format(i)] = clock_state
    file["sweep_fullwv/norm/{}".format(i)] = get_norm(clock_state)
    file["sweep_fullwv/sigma_x/{}".format(i)] = get_sigma_x(clock_state)
    
    print("T = {}".format(time_steps))
        

file["sweep_fullwv/clock_energy"] = clock_energy
file["sweep_fullwv/overlaps/composite"] = overlaps["composite"]
file["sweep_fullwv/overlaps/avg"] = overlaps["avg"]
file.close()

T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50


In [7]:
file.close()

In [6]:
n_sites = 6
h_init = 1.0
h_ev = 0.5

#T_list = [20, 25, 30, 35, 40, 45, 50]
T_list = [20, 50]
t_final = 1.0

ham = tfim.tfim_hamiltonian(n_sites, h=h_ev)
pauli = misc.Pauli()

get_norm = lambda state: (np.abs(state)**2).sum(axis=1)
get_clock_energy = lambda state, clock: (state.ravel().conj().dot(clock.toarray().dot(state.ravel())) 
                                         / (np.abs(state)**2).sum())
get_sigma_x = lambda state: calc.ev_local(state, pauli.X)

file = h5py.File("{}/ClockV3/everything_tf1N{}.h5".format(data_dir, n_sites), "a")

for D in [2, 3, 4, 5, 6]:
    overlaps = {"avg": [], "composite": []}
    clock_energy = []
    for i, time_steps in enumerate(T_list):
        t = np.linspace(0.0, t_final, time_steps + 1)
        dt = t[1] - t[0]
        # Calculate exact evolution
        exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)

        clock = ed.construct_sparse_clock(ham, dt, time_steps)
        clock_state = np.load("{}/ClockV3/final_dense/sweep_mpsD{}_N{}M{}.npy".format(data_dir, D, n_sites, time_steps))
        time_overlap = calc.time_overlap(exact_state, clock_state)
        overlaps["avg"].append(time_overlap.mean())
        overlaps["composite"].append(calc.overlap(exact_state, clock_state))
        clock_energy.append(get_clock_energy(clock_state, clock))

        file["sweepfixed2_mpsD{}/overlaps/time/{}".format(D, i)] = time_overlap
        file["sweepfixed2_mpsD{}/state/{}".format(D, i)] = clock_state
        file["sweepfixed2_mpsD{}/norm/{}".format(D, i)] = get_norm(clock_state)
        file["sweepfixed2_mpsD{}/sigma_x/{}".format(D, i)] = get_sigma_x(clock_state)

        print("T = {}".format(time_steps))


    file["sweepfixed2_mpsD{}/clock_energy".format(D)] = clock_energy
    file["sweepfixed2_mpsD{}/overlaps/composite".format(D)] = overlaps["composite"]
    file["sweepfixed2_mpsD{}/overlaps/avg".format(D)] = overlaps["avg"]

file.close()

T = 20
T = 50
T = 20
T = 50
T = 20
T = 50
T = 20
T = 50
T = 20
T = 50


In [3]:
n_sites = 6
h_init = 1.0
h_ev = 0.5

#T_list = [20, 25, 30, 35, 40, 45, 50]
T_list = [50]
t_final = 1.0

ham = tfim.tfim_hamiltonian(n_sites, h=h_ev)
pauli = misc.Pauli()

get_norm = lambda state: (np.abs(state)**2).sum(axis=1)
get_clock_energy = lambda state, clock: (state.ravel().conj().dot(clock.toarray().dot(state.ravel())) 
                                         / (np.abs(state)**2).sum())
get_sigma_x = lambda state: calc.ev_local(state, pauli.X)

file = h5py.File("{}/ClockV3/everything_tf1N{}.h5".format(data_dir, n_sites), "a")

for D in [2, 3, 4, 5]:
    overlaps = {"avg": [], "composite": []}
    clock_energy = []
    for i, time_steps in enumerate(T_list):
        t = np.linspace(0.0, t_final, time_steps + 1)
        dt = t[1] - t[0]
        # Calculate exact evolution
        exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)

        clock = ed.construct_sparse_clock(ham, dt, time_steps)
        clock_state = np.load("{}/ClockV3/final_dense/sweep_bothways_mpsD{}_N{}M{}.npy".format(data_dir, D, n_sites, time_steps))
        time_overlap = calc.time_overlap(exact_state, clock_state)
        overlaps["avg"].append(time_overlap.mean())
        overlaps["composite"].append(calc.overlap(exact_state, clock_state))
        clock_energy.append(get_clock_energy(clock_state, clock))

        file["sweepbothways_mpsD{}/overlaps/time/{}".format(D, i)] = time_overlap
        file["sweepbothways_mpsD{}/state/{}".format(D, i)] = clock_state
        file["sweepbothways_mpsD{}/norm/{}".format(D, i)] = get_norm(clock_state)
        file["sweepbothways_mpsD{}/sigma_x/{}".format(D, i)] = get_sigma_x(clock_state)

        print("T = {}".format(time_steps))


    file["sweepbothways_mpsD{}/clock_energy".format(D)] = clock_energy
    file["sweepbothways_mpsD{}/overlaps/composite".format(D)] = overlaps["composite"]
    file["sweepbothways_mpsD{}/overlaps/avg".format(D)] = overlaps["avg"]

file.close()

T = 50
T = 50
T = 50
T = 50


In [3]:
n_sites = 6
h_init = 1.0
h_ev = 0.5

T_list = [20, 25, 30, 35, 40, 45, 50]
t_final = 1.0

ham = tfim.tfim_hamiltonian(n_sites, h=h_ev)
pauli = misc.Pauli()

get_norm = lambda state: (np.abs(state)**2).sum(axis=1)
get_clock_energy = lambda state, clock: (state.ravel().conj().dot(clock.toarray().dot(state.ravel())) 
                                         / (np.abs(state)**2).sum())
get_sigma_x = lambda state: calc.ev_local(state, pauli.X)

file = h5py.File("{}/ClockV3/everything_tf1N{}.h5".format(data_dir, n_sites), "a")

for D in [2, 3, 4, 5, 6]:
    overlaps = {"avg": [], "composite": []}
    clock_energy = []
    for i, time_steps in enumerate(T_list):
        t = np.linspace(0.0, t_final, time_steps + 1)
        dt = t[1] - t[0]
        # Calculate exact evolution
        exact_state, obs = tfim.tfim_exact_evolution(n_sites, t[-1], time_steps, h0=h_init, h=h_ev)

        clock = ed.construct_sparse_clock(ham, dt, time_steps)
        clock_state = np.load("{}/ClockV3/final_dense/sweep_oneterm_oneepoch_mpsD{}_N{}M{}.npy".format(data_dir, D, n_sites, time_steps))
        time_overlap = calc.time_overlap(exact_state, clock_state)
        overlaps["avg"].append(time_overlap.mean())
        overlaps["composite"].append(calc.overlap(exact_state, clock_state))
        clock_energy.append(get_clock_energy(clock_state, clock))

        file["sweep_oneterm_mpsD{}/overlaps/time/{}".format(D, i)] = time_overlap
        file["sweep_oneterm_mpsD{}/state/{}".format(D, i)] = clock_state
        file["sweep_oneterm_mpsD{}/norm/{}".format(D, i)] = get_norm(clock_state)
        file["sweep_oneterm_mpsD{}/sigma_x/{}".format(D, i)] = get_sigma_x(clock_state)

        print("T = {}".format(time_steps))


    file["sweep_oneterm_mpsD{}/clock_energy".format(D)] = clock_energy
    file["sweep_oneterm_mpsD{}/overlaps/composite".format(D)] = overlaps["composite"]
    file["sweep_oneterm_mpsD{}/overlaps/avg".format(D)] = overlaps["avg"]

file.close()

T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50
T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50
T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50
T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50
T = 20
T = 25
T = 30
T = 35
T = 40
T = 45
T = 50
